# Finding the volume of a Pareto frontier
Given a Pareto front, calculates the volume "underneath" the frontier (towards the ideal solution)

### Pre-processing

In [46]:
# similar steps to other ipynb.
# except instead of trying to come up with exact volume computation,
# just simulate. And see how many points (simulations) it takes to stabilize (to get vol)

In [47]:
import pandas as pd
import numpy as np

In [48]:
# Get the solutions that define the pareto frontier.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

sols = pd.read_csv("solutionSets/3d/OptimalSolutions_2014XchgRate.csv")

In [49]:
# Hard-coded manipulations to tidy-up sols

sols = sols.drop(["SolutionIndex"],axis=1) #remove solution index column

In [50]:
# Get list of objectives from the dataframe

objs = sols.columns.tolist()

In [51]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the sols dataframe

objSenses = {}

In [52]:
# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 1 # all objectives are max for curr example

In [53]:
# Get the objectives' bounds

objBounds = {}
for obj,sense in objSenses.items():
    objBounds[obj] = [sols[obj].min(),sols[obj].max()]

In [54]:
# Get each objective's ideal value

idealObjVals = {}
for obj,bounds in objBounds.items():
    idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max

In [55]:
# Get each objective's worst value

worstObjVals = {}
for obj,bounds in objBounds.items():
    worstObjVals[obj] = bounds[not(objSenses[obj])]

In [56]:
# Compute the non-normalized volume of the objective space

objSpaceVolume_nonNorm = 1
for obj,bounds in objBounds.items():
    objSpaceVolume_nonNorm *= bounds[1] - bounds[0]

In [57]:
# Normalize the objective space by converting each value to the relative achievement along its axis:

#   distance from the worst case value
# ---------------------------------------
# total distance spanned by the objective

# Objective space is now in [0,1]^N (N is number of objectives)


for obj,bounds in objBounds.items():
    sols[obj] = sols[obj].apply(
        lambda x: \
        abs(x - worstObjVals[obj]) / \
        (bounds[1]-bounds[0]))

### Define hyperrectangle class

In [67]:
class HyperRectangle_Origin:
    """ A class to manufacture hyperrectangle (N-dimensional rectangle) objects
    in which all dimensions live in positive real numbers (R^+)"""

    def __init__(self, farCorner):
        """ Initialize hyperrectangle with extreme points at the origin and farCorner"""
        self.n = len(farCorner)
        self.origin = np.zeros(self.n)
        self.farCorner = farCorner
        self.volume = np.prod(farCorner)
    
    def __str__(self):
        result = "(" + str(self.farCorner[0])
        for coord in self.farCorner[1:]:
            result += ", " + str(coord)
        return  result + ")" 
        

    def contains(self, point):
        """ Determine whether point lies within the hyperrectangle """
        if (np.array(point) < 0).any():
            return False
        else:
            return (np.array(point) <= self.farCorner).all()

### Begin volume computation

In [103]:
# Number of simulations to perform
 
N = np.arange(100,10000,100)

In [104]:
# Create dataframe to hold num-simulations - volume relationship

volsim = pd.DataFrame(index=N, columns=["Volume"])

In [105]:
# Instantiate series of hyperrectangles

hrects = []

In [106]:
# To the frontier, add a hyperrectangle for each solution

for idx, row in sols.iterrows():
    hrects.append(HyperRectangle_Origin(row[objs].tolist()))

In [107]:
# Store the number of dimensions

dims = hrects[0].n

In [110]:
# Compute volume by checking number of points that fall within any of the rectangles
for idx,row in volsim.iterrows():
    numSim = idx
    numWithin = 0
    for n in range(numSim):
        # see if a random point is within the frontier
        rand = np.random.rand(dims)
        numWithin += len([hrect for hrect in hrects if hrect.contains(rand)])>0
    volsim.set_value(idx,"Volume",numWithin/numSim)

In [111]:
volsim

,Volume
10,0.6
20,0.5
30,0.4333333
40,0.65
50,0.64
60,0.55
70,0.6285714
80,0.5625
90,0.5777778
